<a href="https://colab.research.google.com/github/MatheusPiassiC/redes_neurais/blob/main/ann_exerc_02_cat_or_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso: Redes Neurais e Deep Learning

Prof. Denilson Alves Pereira
https://sites.google.com/ufla.br/denilsonpereira/
Departamento de Ciência da Computação -
Instituto de Ciências Exatas e Tecnológicas -
Universidade Federal de Lavras

# Atividade Prática 02

**Instruções:**
1. Siga os passos indicados em cada célula abaixo para completar a atividade.
2. Você deve inserir código somente entre as linhas marcadas com **INICIE O CÓDIGO AQUI** e **TERMINE O CÓDIGO AQUI**. Há uma indicação de quantas linhas de código são necessárias.
3. Em alguns pontos, confira o resultado esperado conforme marcado com **SAÍDA ESPERADA**.

**Tempo estimado para execução**: 1 hora

Versão: Junho, 2021

## O Problema a ser Resolvido

O objetivo da atividade é elaborar uma rede neural para predizer se o animal de uma imagem é um gato ou um cachorro. <br>

Será usado um dataset contendo 8.000 imagens para treinamento e 4.000 para teste, sendo a metade delas de gatos e a outra metade, de cachorros. As imagens têm tamanhos diversos e estão no formato RGB.

Você vai praticar as seguintes habilidades:
- Ler arquivos de imagens e convertê-los para vetores de pixels
- Utilizar um gerador de dados aumentados para expandir o conjunto de imagens para treinamento da rede
- Configurar uma rede neural simples para um problema de classificação multiclasse de imagens. Embora só existam duas classes, a saída será configurada para predizer "cat" se a imagem é de um gato ou "dog" se a imagem é de um cachorro.

É importante destacar que a arquitetura de rede neural utilizada no nesta atividade **não** é a mais indicada para classificar imagens. Para uma melhor eficácia, devem ser adicionadas algumas camadas de Redes Neurais Convolucionais (CNN), que é assunto do próximo curso.

## Pacotes

In [ ]:
import numpy as np # scientific computing
import tensorflow as tf  #  numerical computation using data flow graphs
from tensorflow import keras  # deep learning
from keras_preprocessing.image import ImageDataGenerator, load_img  # generate batches of tensor image data with real-time data augmentation
import pandas as pd # structured data
import matplotlib.pyplot as plt # scientific plotting library
import random  # pseudo-random number generators
import os  # for accessing directory structure

## Inicializações

In [ ]:
IMAGE_WIDTH = 128  # largura da imagem
IMAGE_HEIGHT = 128 # altura da imagem
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT) # tamanho da imagem
IMAGE_CHANNELS = 3  # número de canais RGB da imagem
DIR_TRAIN = "./dataset-cats-and-dogs/training_set/"  # diretório contendo dados de treinamento
DIR_TEST = "./dataset-cats-and-dogs/test_set/"  # diretório contendo dados de teste

## Pré-Processamento dos Dados de Treino e de Teste

Fonte: https://www.kaggle.com/d4rklucif3r/cat-and-dogs

### Treino

In [ ]:
# Read the images from the training directory and generates a pandas structure with the name of each file and the class of the image (cat or dog)
filenames = os.listdir(DIR_TRAIN)
categories = []
for filename in filenames:
    categories.append(filename.split('.')[0]) # category = initial part of the file name

df_train = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df_train.head()

In [ ]:
df_train.tail()

In [ ]:
# number of cats and dogs
df_train['category'].value_counts().plot.bar()

#### Exibe uma amostra

In [ ]:
# Randomly chooses a file and displays its image
sample = random.choice(filenames)
image = load_img(DIR_TRAIN+sample)
plt.imshow(image)
plt.title(sample)

In [ ]:
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)

### Representação de imagens RGB

![imagem-rgb.png](attachment:imagem-rgb.png)

Em imagens coloridas, o padrão RGB (Red, Green, Blue) representa cada pixel como um vetor de três números para os três canais das cores primárias vermelho, verde e azul. Cada número varia de 0 a 255, representando a intensidade da cor correspondente. O três valores combinados formam a cor do pixel. Por exemplo, a cor violeta pode ser representada como 128, 0, 128, uma mistura de intensidade moderada de vermelho e azul, sem o verde.

Já em imagens em escala de cinza (*grayscale*), cada pixel é representado com um único número (entre 0 e 255), o qual determina o quão preto é o pixel (0 é preto e 255 é branco brilhante).

In [ ]:
# convert image to numpy array
data = np.asarray(image)
# summarize shape
print(data.shape)

### Teste

In [ ]:
# Read the images from the test directory and generates a pandas structure with the name of each file and the class of the image (cat or dog)
filenames = os.listdir(DIR_TEST)
categories = []
for filename in filenames:
    categories.append(filename.split('.')[0]) # category = file name

df_test = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df_test.head()

In [ ]:
df_test.tail()

In [ ]:
# number of cats and dogs
df_test['category'].value_counts().plot.bar()

#### Exibe uma amostra

Escolha um arquivo aleatório do conjunto de teste e exiba sua imagem

In [ ]:
# Randomly chooses a file and displays its image
### INICIE O CÓDIGO AQUI ### (4 linhas de código)




### TERMINE O CÓDIGO AQUI ###

### Data Augmentation

*Data Augmentation* (aumento de dados) é uma técnica usada para expandir artificialmente o conjunto de dados de treinamento. Ela ajuda a evitar o problema de *overfitting*, que faz com que o modelo de treinamento fique muito ajustado aos dados usados para o treinamento, e não gera um bom desempenho em outros conjuntos de dados.

No caso das imagens de gatos e cachorros, a ideia é alterar os dados de treino com pequenas transformações nas imagens, de forma a reproduzir as variações que ocorrem quando alguém tira uma foto. Por exemplo, centralização, diferentes escalas (gatos/cachorros grandes e pequenos), a imagem está rotacionada etc. Além disso, o dataset contém imagens de tamanhos diferentes, e precisam ser padronizadas para dar entrada na rede neural.

As abordagens para *data augmentation* alteram os dados de treino de forma a mudar a representação vetorial, mas mantendo o mesmo rótulo de classe. Algumas transformações populares são: grayscales, horizontal flips, vertical flips, random crops, color jitters, translations, rotations, shift, brightness, and zoom..

Fazendo algumas dessas transformações nos dados de treino, é possível multiplicar o número de exemplos de treino, de forma a criar um modelo mais robusto.

Tutorial para configurar *data augmentation* de imagens em Keras: https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/

### Generador de Treino

Documentação: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

Este exemplo foi adaptado de https://www.kaggle.com/uysimty/keras-cnn-dog-or-cat-classification. <br>

In [ ]:
batch_size = 15
train_datagen = ImageDataGenerator(
    rotation_range = 15,
    rescale = 1./255,
    shear_range = 0.1,
    zoom_range = 0.2,
    horizontal_flip = True,
    width_shift_range = 0.1,
    height_shift_range = 0.1
)

# Takes the dataframe and the path to a directory and generates batches.
# The generated batches contain augmented/normalized data.
train_generator = train_datagen.flow_from_dataframe(
    df_train,
    DIR_TRAIN,
    x_col = 'filename',
    y_col = 'category',
    target_size = IMAGE_SIZE,
    class_mode = 'categorical',
    batch_size = batch_size
)

### Generador de Teste

Um gerador de dados pode também ser usado para especificar o dataset de validação e o dataset de teste. Normalmente, uma instância separada de ImageDataGenerator é criada para que se possa configurar a mesma escala de pixels que aquela usada na instância ImageDataGenerator do dataset de treinamento, mas ela não usará *data augmentation*. Isso ocorre porque *data augmentation* é somente usada como uma técnica para expandir artificialmente o dataset de treinamento, para se melhorar o desempenho de um dataset menor.

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    df_test,
    DIR_TEST,
    x_col = 'filename',
    y_col = 'category',
    target_size = IMAGE_SIZE,
    class_mode = 'categorical',
    batch_size = batch_size
)

### Exemplo das Imagens Aumentadas

In [ ]:
df_example = df_train.sample(n=1).reset_index(drop=True)
example_generator = train_datagen.flow_from_dataframe(
    df_example,
    DIR_TRAIN,
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical'
)

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(0, 15):
    plt.subplot(5, 3, i+1)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

## Definição do Modelo

Crie um modelo em Keras com a seguinte configuração:
- Camada de entrada: no formato dos dados de entrada do problema
- Camada *Flatten*: camada para achatar o vetor de entrada. No padrão RGB, cada imagem é representada como um vetor de 3 dimensões. Entretanto, cada entrada em uma rede neural é composta por um vetor de apenas uma dimensão. Assim, é necessário "achatar" o vetor da imagam para uma dimensão. Isso é feito pela camada *Flatten*, que cria um vetor de uma dimensão concatenando sequencialmente os valores nos vetores dos três canais da imagem. Desta forma, se o vetor imagem tem o formato (128, 128, 3), por exemplo, o vetor achatado terá dimensão (49.152, 1), o resultado de 128 x 128 x 3.
- Camada 1: 32 neurônios, função de ativação *ReLu*
- Camada 2: 64 neurônios, função de ativação *ReLu*
- Camada 3: 32 neurônios, função de ativação *ReLu*
- Camada 4 (saída): 2 neurônios, função de ativação *Softmax*. Embora tenha apenas duas classes, o problema deve ser modelado como uma classificação multiclasse ("cat" ou "dog"). Assim, a camada de saída deve ter duas unidades e usar a função Softmax.

Documentação: <br>
Keras: https://keras.io/ <br>
Flatten layer: https://keras.io/api/layers/reshaping_layers/flatten/ <br>
Classe *Model*: https://keras.io/api/models/model/ <br>
Funções de ativação disponíveis: https://keras.io/api/layers/activations/

In [ ]:
### INICIE O CÓDIGO AQUI ### (7 linhas de código)







### TERMINE O CÓDIGO AQUI ###

In [ ]:
# Prints a summary of the network, showing its architecture and parameters.
model.summary()

**SAÍDA ESPERADA**: <br>
Confira a configuração da rede e o total de parâmetros = 1,577,154

---

## Compilação do Modelo

Compile o model usando os seguintes parâmetros:
- Função de perda: categorical_crossentropy
- Otimizador: adam
- Métricas: accuracy

Dica 1: use a função *compile*: https://keras.io/api/models/model_training_apis/ <br>
Dica 2: relação de funções de perda: https://www.tensorflow.org/api_docs/python/tf/keras/losses <br>
Dica 3: relação de otimizadores: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers <br>
Dica 4: relação de métricas: https://keras.io/api/metrics/

In [ ]:
### INICIE O CÓDIGO AQUI ### (1 linha de código)

### TERMINE O CÓDIGO AQUI ###

## Treinamento do Modelo

Ajusta o modelo aos dados de treinamento.
Devem ser fornecidos os dados de treinamento e o número de épocas (iterações). Uma época é composta por uma única passagem por todos os exemplos do conjunto de treino.

Efetue o treinamento do modelo usando 3 épocas com os dados aumentados do gerador de treino *train_generator*.

Dica: use a função *fit*: https://keras.io/api/models/model_training_apis/

In [ ]:
### INICIE O CÓDIGO AQUI ### (1 linha de código)

### TERMINE O CÓDIGO AQUI ###
print(history.history)  # print per-epoch timeseries of metrics values

**SAÍDA ESPERADA**: <br>
Na época 3, tem-se o seguinte resultado (aproximado): <br>
{'loss': [0.8931004405021667, 0.6976273059844971, 0.6935341954231262], 'accuracy': [0.5130000114440918, 0.5195000171661377, 0.5037500262260437]}

---

## Avaliação do Modelo

Avalie o desempenho da rede no conjunto de teste usando o gerador de dados aumentados *test_generator*.

Dica: use a função *evaluate*: https://keras.io/api/models/model_training_apis/

In [ ]:
### INICIE O CÓDIGO AQUI ### (1 linha de código)

### TERMINE O CÓDIGO AQUI ###
print("Loss: %.2f" % loss, "\nAccuracy: %.2f" % acc)

**SAÍDA ESPERADA**: <br>
Valores aproximados:

Loss: 0.69 <br>
Accuracy: 0.50

---

## Predição

Apresente a predição do conjunto de teste usando o gerador de dados aumentados *test_generator*.

Documentação: <br>
Função *predict*: https://keras.io/api/models/model_training_apis/ <br>
Função *argmax*: https://numpy.org/doc/stable/reference/generated/numpy.argmax.html

Na classificação multiclasse, *model.predict* retorna um vetor de probabilidades para cada classe. A função *argmax* retorna o índice do vetor com a maior probabilidade, neste caso, 0 para gato e 1 para cachorro.

In [ ]:
### INICIE O CÓDIGO AQUI ### (1 linha de código)

### TERMINE O CÓDIGO AQUI ###
preds = [np.argmax(x, axis=-1) for x in predictions]
print("Predictions: ", ["cat" if x == 0 else "dog" for x in preds[:30]]) # print the first 30 predictions
print("\nCorrect:     ", [x for x in df_test['category'][:30]])

**SAÍDA ESPERADA**: <br>
Valores aproximados: <br>
Predictions:  ['dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog']

Correct:      ['dog', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'dog', 'cat', 'dog', 'cat', 'dog', 'dog', 'cat', 'cat', 'cat', 'cat', 'dog', 'cat', 'dog', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat']

---

# Fim

Esses foram os passos para criação de uma rede neural simples para classificação de imagens. Lembrando que para se obter uma melhor eficácia, devem ser adicionadas algumas camadas de Redes Neurais Convolucionais (CNN), que é assunto do próximo curso.

-------------------------------------------